# 프롬프트 자동 완성기

## 개요:
프롬프트를 자동으로 완성해주며 LLM으로부터 좋은 답변을 받을 수 있도록 부족한 문장을 완성해 주는 도구.

<br>

## 제작 목적:
사용자의 제품 사용을 유도하고 흥미를 불러일으키는 데 있다.
1) 사용자 문제:
  - 사용자의 질문이 구체적이지 못하고 불완전하다.
  - 멀티턴 사용자의 비율이 적다.

2) 해결 방법
  - 사용자가 프롬프트를 잘 쓸 수 있도록 안내하거나 보조하는 장치를 만든다.

3) 구현 아이디어
  - AI와의 대화를 원활히 진행하고, 사용 가치와 효용성을 극대화시킬 수 있는 프롬프트 자동완성기를 제작한다.

우리 Biblo 솔루션을 사용하는 사용자의 질의 의도는 '학술 정보 검색'이다.
즉, 사용자는 원하는 학술 정보를 얻기 위해 우리의 서비스를 이용한다.

<br>

## 특징: 단계적 추론
1) 사용자 질의가 들어오면, 카테고리 내에서 의도 분류
  
    ex) 도서 검색, 논문 검색, 도서관 콘텐츠 이용,

2) 의도에 맞게 상세한 프롬프트 작성



# 종속성 설치

In [ ]:
!pip install --upgrade --quiet  langchain langchain-core langchain-community langchain-text-splitters langchain-milvus langchain-openai bs4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.9 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade--quiet tokenizers


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --upgrade--quiet


In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 32.4 MB/s eta 0:00:00


# LLM 모델 설치

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 1. 모델 ID 설정
base_model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'

# 2. BitsAndBytesConfig 설정
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True, # 4bit quantization으로 모델을 불러와 메모리 소모를 줄임
)

# 3. 모델 로드
model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

In [ ]:
# 4. 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=512,  # 최대 토큰 길이
    padding_side="left",   # 입력 패딩 방향
    add_eos_token=True     # EOS 토큰 추가
)

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
# LLM 파이프라인 설정 (LangChain과 통합)
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=500
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

Device set to use cuda:0
<ipython-input-6-0e279de07470>:10: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


# 질의 증강 프롬프트 엔지니어링

In [ ]:
from langchain.prompts import PromptTemplate


PROMPT_QUERY_AUGMENTED = """
---As a scholar, I am looking for information on the latest scholarly research trends.
Your role as an "Academic Research Information Services Specialist" is to "generate new query" that enhance the scholarly expertise in my questions and help me find scholarly resources.
My question is "{question}".

---Then follow the instructions.
<Rules>
After understanding the intent of my question, please "generate new query" in detail to make it professional and complete.
Always create roles, situations, and tasks within "generate new query", if possible.
</Rules>

<Strong Rules>
"generate new query" should be in half-speech form of "Korean(반말)".
"generate new query" should be "limited to two sentences".
</Strong Rules>
---
#[End]

"""

In [ ]:
from langchain_core.prompts import PromptTemplate
# PromptTemplate 설정
prompt = PromptTemplate(
    template=PROMPT_QUERY_AUGMENTED,
    input_variables=["question"]
)

In [ ]:
# 테스트 입력
query = "LLM과 Transformer의 차이를 알고 싶어."
formatted_prompt = prompt.format(question=query)
print("Formatted Prompt:\n", formatted_prompt)

Formatted Prompt:
 
---As a scholar, I am looking for information on the latest scholarly research trends.
Your role as an "Academic Research Information Services Specialist" is to "generate new query" that enhance the scholarly expertise in my questions and help me find scholarly resources.
My question is "LLM과 Transformer의 차이를 알고 싶어.".

---Then follow the instructions.
<Rules>
After understanding the intent of my question, please "generate new query" in detail to make it professional and complete.
Always create roles, situations, and tasks within "generate new query", if possible.
</Rules>

<Strong Rules>
"generate new query" should be in half-speech form of "Korean(반말)". 
"generate new query" should be "limited to two sentences".
</Strong Rules>
---
#[End]




In [ ]:
response = llm(formatted_prompt)

print("LLM Response:\n", response)

LLM Response:
 
---As a scholar, I am looking for information on the latest scholarly research trends.
Your role as an "Academic Research Information Services Specialist" is to "generate new query" that enhance the scholarly expertise in my questions and help me find scholarly resources.
My question is "LLM과 Transformer의 차이를 알고 싶어.".

---Then follow the instructions.
<Rules>
After understanding the intent of my question, please "generate new query" in detail to make it professional and complete.
Always create roles, situations, and tasks within "generate new query", if possible.
</Rules>

<Strong Rules>
"generate new query" should be in half-speech form of "Korean(반말)". 
"generate new query" should be "limited to two sentences".
</Strong Rules>
---
#[End]

---

### generate new query

"LLM과 Transformer의 차이점을 연구한 최신 연구 논문과 논문 리뷰를 제공해 주세요. LLM과 Transformer의 기능, 성능, 적용 분야 등을 비교하여 학문적 지식을 강화해 주세요."

### Explanation
1. **Query Intent**: The user wants to know the differences between LLM (La

In [ ]:
# 사용자 질의 테스트
query = "시간복잡도가 애플리케이션의 사용에 영향을 미치지 알고싶어"
# Format the prompt template with the user's query
formatted_prompt = prompt.format(question=query)
response = llm(formatted_prompt)

print("LLM Response:\n", response)

LLM Response:
 
---As a scholar, I am looking for information on the latest scholarly research trends.
Your role as an "Academic Research Information Services Specialist" is to "generate new query" that enhance the scholarly expertise in my questions and help me find scholarly resources.
My question is "시간복잡도가 애플리케이션의 사용에 영향을 미치지 알고싶어".

---Then follow the instructions.
<Rules>
After understanding the intent of my question, please "generate new query" in detail to make it professional and complete.
Always create roles, situations, and tasks within "generate new query", if possible.
</Rules>

<Strong Rules>
"generate new query" should be in half-speech form of "Korean(반말)". 
"generate new query" should be "limited to two sentences".
</Strong Rules>
---
#[End]

---

### Generate New Query

안녕하세요, 저는 연구자로서 시간복잡도가 애플리케이션의 사용에 영향을 미치지 않는지 궁금합니다. 이를 위해 시간복잡도가 애플리케이션의 성능에 미치는 영향을 이해하는 것이 중요하다는 것을 알고 싶습니다.

---

### Explanation
"안녕하세요, 저는 연구자로서 시간복잡도가 애플리케이션의 사용에 영향을 미치지 않는지 궁금합니다" 
* "안녕하세요" 

In [ ]:
# 사용자 질의 테스트
query = "조선시대 전기 백자와 청자의 사용에 대해 정리해줘."
# Format the prompt template with the user's query
formatted_prompt = prompt.format(question=query)
response = llm(formatted_prompt)

print(response)


---As a scholar, I am looking for information on the latest scholarly research trends.
Your role as an "Academic Research Information Services Specialist" is to "generate new query" that enhance the scholarly expertise in my questions and help me find scholarly resources.
My question is "조선시대 전기 백자와 청자의 사용에 대해 정리해줘.".

---Then follow the instructions.
<Rules>
After understanding the intent of my question, please "generate new query" in detail to make it professional and complete.
Always create roles, situations, and tasks within "generate new query", if possible.
</Rules>

<Strong Rules>
"generate new query" should be in half-speech form of "Korean(반말)". 
"generate new query" should be "limited to two sentences".
</Strong Rules>
---
#[End]

---

1. **Contextual Understanding**
The user is seeking information about the usage and significance of electric porcelain and ceramic in the Joseon Dynasty.

2. **Research Objective**
To provide a comprehensive overview of the historical context 

In [ ]:
# 사용자 질의 테스트
query = "거시경제학과 케인즈의 관계를 알려줘."
# Format the prompt template with the user's query
formatted_prompt = prompt.format(question=query)
response = llm(formatted_prompt)

print(response)


---As a scholar, I am looking for information on the latest scholarly research trends.
Your role as an "Academic Research Information Services Specialist" is to "generate new query" that enhance the scholarly expertise in my questions and help me find scholarly resources.
My question is "거시경제학과 케인즈의 관계를 알려줘.".

---Then follow the instructions.
<Rules>
After understanding the intent of my question, please "generate new query" in detail to make it professional and complete.
Always create roles, situations, and tasks within "generate new query", if possible.
</Rules>

<Strong Rules>
"generate new query" should be in half-speech form of "Korean(반말)". 
"generate new query" should be "limited to two sentences".
</Strong Rules>
---
#[End]

--- As an Academic Research Information Services Specialist, I will generate a new query to provide you with the latest scholarly research trends related to the topic you are interested in. Here is the query:

"저는 거시경제학과 케인즈의 관계를 연구하고자 하는 학자입니다. 현재 거시경제학과 